# Capstone - Finding the best region for a restaurant in Toronto

This notebook is used mainly for the capstone project of Eduardo Faria in the coursera course IBM Data Science.

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

The objective of this project is to identify the best region (cluster / neighborhoods) to open a restaurant in Toronto. To find the best place, we will look for the region with the highest concentration of restaurants, which indicates a good area for this type of business. This information is important for the entrepreneur who wants to open a business to be successful and is directed to those who want to open a restaurant in Toronto.

## Data <a name="data"></a>

To solve that problem we will use the venue information obtained from Foursquare location data. From Foursquare  we get information about the venues in each place for example coffee shop, restaurants, Gym.

Then, the neighborhood information from Toronto available at wikipedia is used to segmented and clustered Toronto's neighborhoods: "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M".

In the treatment of the data it has been used the libraries BeautifulSoup (scrap data from web), Geopy (get latitude and longitude), Folium (render maps), Sklearn KMeans(clustering) and, of course, numpy, pandas and matplotlib.

#### Following is an example of the information from wikipedia about Toronto Postal Codes / Neighborhoods:

<img src="example_wikipedia_toronto.PNG"/>

#### This is an example of the data from the Foursquare data after being cleaned and placed in a dataframe

<img src="example_foursquare_data.PNG"/>

## Read data from web and prepare dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Install package BeautifulSoup
#!pip install beautifulsoup4
#!pip install lxml

In [3]:
import requests
from bs4 import BeautifulSoup

### Read postal code data from wikipedia
Uses requests and BeautifulSoup to get the data from webpage.

In [4]:
# Site with postal code data of Toronto
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())

In [5]:
site_table = soup.find('table', class_='wikitable')
#print(site_table.prettify())

### Create a dictionary and clean the data  
- Ignore cells with a borough that is 'Not assigned'.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
# Read Postal Code table from wikipedia
postalCode_list = []
borough_list = []
neighborhood_list = []
line_num = 0
for line in site_table.find_all('tr'):
    #Proccess each data row
    col_item = 0
    postalCode = ''
    borough = ''
    neighborhood = ''
    #Read each column
    for col in line.find_all('td'):
        if col_item == 0:
            postalCode = col.text.strip(' \t\n\r')
        elif col_item == 1:
            borough = col.text.strip(' \t\n\r')
        elif col_item == 2:
            neighborhood = col.text.strip(' \t\n\r')
        col_item += 1
    
    line_num += 1
    
    #ignore Borough 'Not assigned'
    if (borough != 'Not assigned') and (len(borough) > 0):
        #Set neighborhood the same as the borough, if cell has a borough but a Not assigned neighborhood
        if (not neighborhood):
            neighborhood = borough
        #Change multiple neighborhood separator '/' per ', ' if needed
        neighborhood = neighborhood.replace(' /', ',')
        #Append the row data
        postalCode_list.append(postalCode)
        borough_list.append(borough)
        neighborhood_list.append(neighborhood)
data = {
    'PostalCode': postalCode_list,
    'Borough': borough_list,
    'Neighborhood': neighborhood_list
}

### Create pandas DataFrame

In [7]:
# Creating pandas Dataframe
neighborhoods = pd.DataFrame(data)
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Merge rows with same postal code

In [8]:
# Merge rows with same postal code
for i, row in neighborhoods.iterrows():
    # Go through the list and compare 'row' Postal Code with 'other_row' Postal Code
    for other_i, other_row in neighborhoods.iterrows():
        # if same Postal Code, merge data
        if (other_row.PostalCode == row.PostalCode) and (other_i != i) and (other_i > i):
            print('duplicate: ', i, ' ', other_i)
            # combine data in column neighborhood
            neighborhoods.iloc[i].Neighborhood = neighborhoods.iloc[i].Neighborhood + ', ' + neighborhoods.iloc[other_i].Neighborhood
            print(neighborhoods.iloc[i].Neighborhood)
            # delete duplicated postal code row
            neighborhoods.drop(other_i, inplace=True)
neighborhoods.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Shape of the dataframe with neighborhood information

In [9]:
neighborhoods.shape

(103, 3)

## Getting Latitudes and Longitudes

### Note: Geocoder lib did not work well, then csv file with latitude and longitude is used

In [10]:
# install Geocoder API to get Latitude and longitude
#!pip install geocoder
#import geocoder

# initialize variable to None
#lat_lng_coords = None

# loop until get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

### Read Coordinates from csv

In [11]:
coords = pd.read_csv('https://cocl.us/Geospatial_data')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Include latitude and longitude in the main Dataframe 'neighborhoods' 

In [12]:
# Find coords for each neigborhoods
latitude = []
longitude = []
for i, neighborhood in neighborhoods.iterrows():
    postalCode = neighborhood.PostalCode 
    coord = coords.loc[coords['Postal Code'] == postalCode]
    latitude.append(coord.Latitude.item())
    longitude.append(coord.Longitude.item())
# Include Latitude and Longitude in the Dataframe
neighborhoods['Latitude'] = latitude
neighborhoods['Longitude'] = longitude
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Methodology <a name="methodology"></a>

In this project we will direct detecting areas of Toronto that have high restaurant density. We will limit our analysis to boroughs that contain the word Toronto.

A map of Toronto Neighborhoods is plot with Folium Complete and simplified.

In first step we have collected the venue data for each neighborhood of Toronto using Foursquare.

After we check how many venues were returned for each neighborhood.

Clustering technique using K-means is used to group the neighborhoods.

The most common revenue for each cluster is found.

Finaly we select the cluster with more restaurants in first more commom revenue.

## Analysis <a name="analysis"></a>

First let's count the **number of restaurants in every area candidate**:

### Instaling Libraries

In [13]:
# install geopy library
#!pip install geopy
# import Nominatim - convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

In [14]:
# install folium library
#!pip install folium
# import folium library
import folium

### Find latitude and longitude of Toronto

In [15]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer") #user agent need to be defined 
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_lng = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_lat, toronto_lng))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top.

In [16]:
# install folium library
#!pip install folium
# import folium library
import folium

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### To simplify, the work will be only with boroughs that contain the word Toronto

In [18]:
# Select boroughs that contain the word Toronto
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
# Drop column PostalCode
toronto_data.drop('PostalCode', axis=1, inplace=True)
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


In [19]:
toronto_data.shape

(39, 4)

#### Visualize Neighborhoods in the simplified data

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = '4Z0NPJSEVQB5WIMPULIGF3TIXQJYGSSH55DZJZEZE4MHRWTV' # Foursquare ID
CLIENT_SECRET = 'D42Z4EN5FK33OZSH5GOI3LC2XE5TWSVS10KUJGDBDKSRGUT4' # Foursquare Secret
VERSION = '20180604'

#### Function to repeat the same process to all the neighborhoods in Toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood and create a new dataframe called toronto_venues

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street


#### Check the size of the resulting dataframe

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

#### Check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

#### How many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

### Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

#### Dataframe size

In [ ]:
toronto_onehot.shape

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Size

In [ ]:
toronto_grouped.shape

#### Each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### 5 most common venues in Dataframe

Function to sort the venues in descending order

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster

#### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Results and Discussion <a name="results"></a>

As we could see above, the cluster 3 is the one with more concentration of restaurants as more common revenue, indicating that it is appropriated to open a restaurant.

#### Below we list the Neighborhoods in the cluster 3

In [ ]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
cluster3[['Neighborhood']]

## Conclusion <a name="conclusion"></a>

Purpose of this project is to identify the best region (cluster / neighborhoods) to open a restaurant in Toronto city. 

To find the best place, we have collected the venue data for each neighborhood of Toronto using Foursquare. Then, checked how many venues were returned for each neighborhood. The clustering technique using K-means was used to group the neighborhoods and find the most common in each cluster. Finally we selected the cluster with more restaurants in first more commom revenue.

The results as seen above show that the best place to open a restaurant in Toronto is indicated in Cluster 3, representing the neighborhoods listed in the section above.